In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
df=pd.read_csv('../3lab/day.csv')

In [3]:
df.describe()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000
mean,366.000000,2.496580,0.500684,6.519836,0.028728,2.997264,0.683995,1.395349,0.495385,0.474354,0.627894,0.190486,848.176471,3656.172367,4504.348837
std,211.165812,1.110807,0.500342,3.451913,0.167155,2.004787,0.465233,0.544894,0.183051,0.162961,0.142429,0.077498,686.622488,1560.256377,1937.211452
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.059130,0.079070,0.000000,0.022392,2.000000,20.000000,22.000000
25%,183.500000,2.000000,0.000000,4.000000,0.000000,1.000000,0.000000,1.000000,0.337083,0.337842,0.520000,0.134950,315.500000,2497.000000,3152.000000
50%,366.000000,3.000000,1.000000,7.000000,0.000000,3.000000,1.000000,1.000000,0.498333,0.486733,0.626667,0.180975,713.000000,3662.000000,4548.000000
75%,548.500000,3.000000,1.000000,10.000000,0.000000,5.000000,1.000000,2.000000,0.655417,0.608602,0.730209,0.233214,1096.000000,4776.500000,5956.000000
max,731.000000,4.000000,1.000000,12.000000,1.000000,6.000000,1.000000,3.000000,0.861667,0.840896,0.972500,0.507463,3410.000000,6946.000000,8714.000000


In [4]:
df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [5]:
X=df[['hum','temp','windspeed']]
y=df['cnt']

In [6]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(len(X.columns), activation='selu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam',
              loss='mean_squared_error'
)

model.fit(X_train, y_train, verbose=0, batch_size=10, epochs=100)

y_pred = model.predict(X_test)

print(f'R2_score: {round(r2_score(y_test, y_pred), 2)}')
print(f'MSE: {round(mean_squared_error(y_test, y_pred), 2)}')
print(f'MAE: {round(mean_absolute_error(y_test, y_pred), 2)}')

R2_score: 0.34
MSE: 2489095.98
MAE: 1292.66


In [8]:
params = [
    {
        'layers': [
            tf.keras.layers.Dense(len(X.columns), activation='relu'),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(1)
        ],
        'optimizer': 'adam',
        'epoch': 100
    },
    {
        'layers': [
            tf.keras.layers.Dense(len(X.columns), activation='selu'),
            tf.keras.layers.Dense(128, activation='selu'),
            tf.keras.layers.Dense(1)
        ],
        'optimizer': 'adam',
        'epoch': 150
    },
    {
        'layers': [
            tf.keras.layers.Dense(len(X.columns), activation='relu'),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1)
        ],
        'optimizer': 'rmsprop',
        'epoch': 200
    },
]

for i, p in enumerate(params):
    model = tf.keras.Sequential(p['layers'])

    model.compile(optimizer=p['optimizer'],
                  loss='mean_squared_error'
    )

    model.fit(X_train, y_train, verbose=0, batch_size=10, epochs=p['epoch'])

    y_pred = model.predict(X_test)
    
    print(f'Parameters #{i+1}')
    print(f'R2_score: {round(r2_score(y_test, y_pred), 2)}')
    print(f'MSE: {round(mean_squared_error(y_test, y_pred), 2)}')
    print(f'MAE: {round(mean_absolute_error(y_test, y_pred), 2)}\n')

Parameters #1
R2_score: 0.23
MSE: 2927010.57
MAE: 1398.03

Parameters #2
R2_score: 0.38
MSE: 2361148.97
MAE: 1288.51

Parameters #3
R2_score: 0.2
MSE: 3016269.52
MAE: 1413.56

